In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score,f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("preprocessed_train.csv")
out = pd.read_csv("preprocessed_test.csv")
df.head()

,index,Popularity,Page content,data-channel,title,raw_content,content_length,avg_word_length,topics,author,year,month,day,hour,min,sec,weekends,img_number,link_number,title_length
0,1,1,"<html><head><div class=""article-info""><span cl...",28,google new open sourc patent pledge We won t s...,googl took stand sort patent lawsuit theater t...,309,5.032787,"apps and software,google,open source,opn pledg...",94,2013,3,28,17,40,55,0,2,18,74
1,2,1,"<html><head><div class=""article-info""><span cl...",7,ballin nfl draft pick get choos their own walk...,you v spend countless hour train american foot...,1360,4.750225,"entertainment,nfl,nfl draft,sports,television",351,2014,5,7,19,15,20,0,2,11,68
2,3,-1,"<html><head><div class=""article-info""><span cl...",31,cameraperson fail deliv slapstick laugh,tire old sport fail news fails let cameraperso...,476,4.841727,"sports,video,videos,watercooler",351,2013,10,11,2,26,50,0,1,13,43
3,4,-1,"<html><head><div class=""article-info""><span cl...",7,nfl star help young fan prove friendship with ...,At pounds all pro nfl star j j watt terror for...,1937,5.089650,"entertainment,instagram,instagram video,nfl,sp...",102,2014,4,17,3,31,43,0,52,16,61
4,6,1,"<html><head><div class=""article-info""><span cl...",21,vine video Is funniest parodi iggi azalea,first thing first hilarious A short parodi vid...,116,4.824074,"entertainment,funny,iggy azalea,music,parody,v...",70,2014,8,11,5,0,18,0,1,11,67


In [3]:
df = df.drop(columns=["index","year","title_length","min","sec","index",'Page content',"raw_content","title","topics"])
df

,Popularity,data-channel,content_length,avg_word_length,author,month,day,hour,weekends,img_number,link_number
0,1,28,309,5.032787,94,3,28,17,0,2,18
1,1,7,1360,4.750225,351,5,7,19,0,2,11
2,-1,31,476,4.841727,351,10,11,2,0,1,13
3,-1,7,1937,5.089650,102,4,17,3,0,52,16
4,1,21,116,4.824074,70,8,11,5,0,1,11
...,...,...,...,...,...,...,...,...,...,...,...
24945,-1,32,308,4.848185,250,4,8,16,0,2,12
24946,-1,28,330,4.924290,5,7,9,1,0,3,23
24947,-1,31,331,5.552941,96,7,10,12,0,15,23
24948,-1,3,436,5.167442,367,4,16,20,0,3,17


In [4]:
out = out.drop(columns=["Id","year","title_length","min","sec","index",'Page content',"raw_content","title","topics"])
out.head(5)


,data-channel,content_length,avg_word_length,author,month,day,hour,weekends,img_number,link_number
0,1949,622,5.220114,243,9,9,19,0,1,30
1,7808,149,4.739437,263,10,31,9,0,3,13
2,1692,168,5.018293,278,6,25,12,0,2,13
3,3579,162,5.581699,211,2,13,3,0,1,15
4,1952,225,5.041096,142,10,3,1,0,1,10


In [8]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
out_std = sc.transform(out)

In [9]:
from sklearn.ensemble import AdaBoostClassifier
AdaBoost = AdaBoostClassifier(n_estimators=10,random_state=10)
AdaBoost.fit(X_train_std,y_train)
y_pred_ada = AdaBoost.predict(X_test_std)
print('Accuracy: %f' % accuracy_score(y_test, y_pred_ada))
print('f1-score: %f' % f1_score(y_test, y_pred_ada))
print(AdaBoost.feature_importances_)


Accuracy: 0.566733
f1-score: 0.552010
[0.2 0.  0.  0.3 0.3 0.  0.  0.1 0.  0.1]


In [10]:
ans_pred = AdaBoost.predict(out_std)
lst = []
for i in ans_pred:
    if (i == 1):
        lst.append(1)
    else:
        lst.append(0)


In [ ]:
ans_sap = pd.DataFrame(lst, columns = ['Popularity'])
ans_sap.to_csv('y_pred.csv',  index =  False)

In [ ]:
tree = DecisionTreeClassifier(criterion='entropy',max_depth =8)
tree.fit(X_train_std,y_train)

y_pred = tree.predict(X_test_std)

print('Accuracy: %f' % accuracy_score(y_test, y_pred))
print('f1-score: %f' % f1_score(y_test, y_pred))
print(tree.feature_importances_)

In [ ]:
lr = LogisticRegression(C = 100.0, random_state = 10, solver = "liblinear")
lr.fit(X_train_std, y_train)http://localhost:8888/notebooks/Desktop/DL_compete/test.ipynb#
y_pred_lr = tree.predict(X_test_std)
print('Accuracy: %f' % accuracy_score(y_test, y_pred_lr))
print('f1-score: %f' % f1_score(y_test, y_pred_lr))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=15, p=4, metric='minkowski')

knn.fit(X_train_std, y_train)
y_pred = knn.predict(X_test_std)
print('Accuracy: %f' % accuracy_score(y_test, y_pred))
print('f1-score: %f' % f1_score(y_test, y_pred))